In [3]:
# https://github.com/binance/binance-public-data/

import os
import requests
import ccxt
import pandas as pd

# 获取Spot交易对
def spot_symbol_fetch(exchange):
    symbol_info = pd.DataFrame(exchange.publicGetExchangeInfo()['symbols'])
    symbol_info = symbol_info[(symbol_info['quoteAsset'] == 'USDT')][['symbol','baseAsset']]
    
    # 过滤杠杆代币
    ends_with_conditions = symbol_info['baseAsset'].str.endswith(('UP', 'DOWN', 'BULL', 'BEAR'))
    exact_conditions = symbol_info['baseAsset'].isin(['JUP', 'BULL', 'BEAR'])
    symbol_info = symbol_info[~(ends_with_conditions & ~exact_conditions)]
    return symbol_info

# 获取U-Margin Swap交易对
def um_symbol_fetch(exchange):
    symbol_info = pd.DataFrame(exchange.fapiPublicGetExchangeInfo()['symbols'])
    symbol_info = symbol_info[(symbol_info['contractType'] == 'PERPETUAL') & (symbol_info['quoteAsset'] == 'USDT')][['symbol']]
    return symbol_info

# 获取Coin-Margin Swap交易对
def cm_symbol_fetch(exchange):
    symbol_info = pd.DataFrame(exchange.dapiPublicGetExchangeInfo()['symbols'])
    symbol_info = symbol_info[symbol_info['contractType'] == 'PERPETUAL'][['symbol']]
    return symbol_info

# def download_binance_data(url, save_path, file_name):
def save_trade_data(url, save_path, file_name):
    # Create the designated folder if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Full path to save the file
    file_path = os.path.join(save_path, file_name)
    
    # Download the file
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"File downloaded and saved to {file_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

#===========开始运行
spot_path = '/Users/zhouyufeng/Documents/crypto_volume_forecast/trade/spot'
um_path = '/Users/zhouyufeng/Documents/crypto_volume_forecast/trade/um'
cm_path = '/Users/zhouyufeng/Documents/crypto_volume_forecast/trade/cm'

# 交易所配置
exchange = ccxt.binance()

# 日期配置
# start = (datetime.datetime.now() + datetime.timedelta(days = -1)).strftime('%Y-%m-%d')
# end = (datetime.datetime.now() + datetime.timedelta(days = -1)).strftime('%Y-%m-%d')
start = '2025-02-10'
end = '2025-02-11'
date_list = pd.date_range(start, end)
date_list = [i.strftime('%Y-%m-%d') for i in date_list]

# 获取永续与现货清单
# spot_symbol_list = sorted(spot_symbol_fetch(exchange)['symbol'])[:2]
# um_symbol_list = sorted(um_symbol_fetch(exchange)['symbol'])[:2]
# cm_symbol_list = sorted(cm_symbol_fetch(exchange)['symbol'])[:2]
spot_symbol_list = ['BTCUSDT', 'ETHUSDT', 'CAKEUSDT']
um_symbol_list = []
cm_symbol_list = []

for symbol_type in ['spot', 'um', 'cm']:
    if symbol_type == 'spot':
        symbol_list = spot_symbol_list
        _ = spot_path
        url_ = 'https://data.binance.vision/data/spot/daily/trades/'
    elif symbol_type == 'um':
        symbol_list = um_symbol_list
        _ = um_path
        url_ = 'https://data.binance.vision/data/futures/um/daily/trades/'
    elif symbol_type == 'cm':
        symbol_list = cm_symbol_list
        _ = cm_path
        url_ = 'https://data.binance.vision/data/futures/cm/daily/trades/'
    for trade_date in date_list:
        # 遍历交易对
        for symbol in symbol_list:
            url = url_ + symbol + '/' + symbol + '-trades-' + trade_date + '.zip'
            save_path = _ + '/' + trade_date + '/'
            file_name = symbol + '-trades-' + trade_date + '.zip'
            print (symbol_type, symbol, trade_date)
            save_trade_data(url, save_path, file_name)


# data reading
# df = pd.read_csv('/Users/zhouyufeng/Documents/crypto_volume_forecast/trade/spot/2025-02-11/ETHUSDT-trades-2025-02-11.zip'
#                  , header = None, compression = 'zip')
# df.columns = ['trade_id', 'price', 'qty', 'quoteQty', 'time', 'isBuyerMaker', 'isBestMatch']
# df['time'] = pd.to_datetime(df['time'], unit='us')

spot BTCUSDT 2025-02-10


PermissionError: [Errno 13] Permission denied: '/Users/zhouyufeng'